In [9]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
#Model 1
#train_input_file = "./submission/train_cleaned.csv"
#test_input_file ="./submission/test_cleaned.csv"
# features =[
# 'ordinal_Overall Qual','Gr Liv Area','Garage Area','Total Bsmt SF',
# '1st Flr SF','Year Built','Year Remod/Add','Full Bath', 'TotRms AbvGrd',
# 'Mas Vnr Area', 'Fireplaces','BsmtFin SF 1','Lot Area','Open Porch SF',
# 'Lot Frontage','Wood Deck SF','Half Bath'
# ]

In [11]:
train_input_file = "./submission/train_cleaned_ord.csv"
test_input_file ="./submission/test_cleaned_ord.csv"

In [12]:
train_df = pd.read_csv(train_input_file)
test_df =pd.read_csv(test_input_file)

In [13]:
column_null_counts = test_df.isnull().sum() *100/ len(test_df) #Filter to find missing data in percetage 
missing_data_columns =column_null_counts[column_null_counts >0] # get only columns where missing data  >0
missing_data_columns.sort_values(ascending=False) # display them in highest missing data at top

Garage Yr Blt    0.113895
dtype: float64

columns_to_drop =['SalePrice','MS SubClass',
                  'Misc Val', 'Mo Sold', 'Yr Sold',
                   'Low Qual Fin SF',
                  'Year Built', 'Mas Vnr Area', 'Bsmt Unf SF','Garage Yr Blt',
                 'Screen Porch',
                  '3Ssn Porch','Lot Frontage']
features =  df_train._get_numeric_data().drop(columns =columns_to_drop).columns                  

In [14]:
# features =[
# 'ordinal_Overall Qual','ordinal_Exter Qual','Gr Liv Area','ordinal_Kitchen Qual','Garage Area',
# 'Garage Cars','Total Bsmt SF','1st Flr SF','ordinal_Bsmt Qual',
# 'Year Built','ordinal_Garage Finish','Year Remod/Add','ordinal_Fireplace Qu',
# 'Full Bath', 'TotRms AbvGrd','Mas Vnr Area', 'Fireplaces','ordinal_Heating QC',
# #'ordinal_Bsmt Exposure','BsmtFin SF 1','ordinal_BsmtFin Type 1','Lot Area','Open Porch SF',
# #'Lot Frontage','Wood Deck SF'
# ] 
#0.33 is the cutoff

In [15]:
features =[
'ordinal_Overall Qual','Gr Liv Area','Garage Area','Total Bsmt SF',
'1st Flr SF','ln_house_age','Year Remod/Add','Full Bath', 'TotRms AbvGrd',
'Mas Vnr Area', 'Fireplaces','BsmtFin SF 1','Lot Area','Open Porch SF',
'Lot Frontage','Wood Deck SF','Half Bath','f_has_open_porch','f_has_wooden_deck',

'ordinal_Exter Qual','ordinal_Kitchen Qual','ordinal_Bsmt Qual','ordinal_Bsmt Exposure',
'ordinal_Fireplace Qu','ln_house_age','ordinal_Electrical',
'f_tot_porches','Enclosed Porch','ordinal_Overall Cond','nominal__Inside',
'Low Qual Fin SF','nominal__FR2','nominal__RRNe',
    
'nominal_garage__Detchd' ,'nominal_garage__N_A'
#'nominal__Y','nominal__GasW','Misc Val','ordinal_Paved Drive'
    

# 'nominal__None','nominal__Norm','nominal__PosN'
#,,'nominal__Feedr.1','nominal__OthW',
] 



In [ ]:
test_df.columns[train_df.columns.str.startswith('nominal_')]

In [ ]:
train_df.corr()[['SalePrice']].sort_values(by='SalePrice',ascending=False)

In [ ]:
apps_core = train_df.corr()[['SalePrice']].sort_values(by='SalePrice',ascending=False)
plt.figure(figsize=(6,12))  # 6 by 12 inch to make plot bigger
sns.heatmap(apps_core, vmin=-1,vmax=1,cmap='coolwarm');

In [ ]:
X =train_df[features]

# columns_to_drop =['SalePrice','ordinal_Garage Finish', 'ordinal_Electrical' ,  'ordinal_Garage Cond ',   'ordinal_Garage Qual' ,     'ordinal_Garage Cond',       'ordinal_Garage Qual',        'ordinal_Garage Finish',       'Garage Yr Blt',          'ordinal_Electrical'  ]
# X = train_df._get_numeric_data().drop(columns =columns_to_drop)
# features =list(X.columns)

y_orig =train_df['SalePrice']
y=np.log(train_df['SalePrice'])
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,stratify=y)
lr =LinearRegression()
lr.fit(X_train,y_train);

In [ ]:
train_score= lr.score(X_train,y_train)
test_score = lr.score(X_test,y_test)
cvs =cross_val_score(lr,X_train,y_train,cv=5).mean()

print(f"\ntrain r2 score ={train_score}, \ncross_val_score= {cvs}\ntest score ={test_score}")

In [ ]:
y_hat =lr.predict(X)
y_hat_orig = np.exp(y_hat)
rmse= np.sqrt(metrics.mean_squared_error(y_orig,y_hat_orig ))
print(f"RMSE (equivalent Kaggle score={rmse}")

### Using the model to now predict the  data in test.csv

In [ ]:
X_kaggle_test =test_df[features]
test_df['SalePrice_log'] =lr.predict(X_kaggle_test)
test_df['SalePrice'] =np.exp(test_df['SalePrice_log'])                    

In [ ]:

test_df[['Id' ,'SalePrice']].to_csv('./submission/ames_pred_01_16_01.csv',index=False)